MNIST Data Exploration

In [36]:

import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import xgboost
from sklearn import metrics

#all the data is under the same folder
import os
print(os.listdir("../mnist_data_exploration"))


['.git', '.gitignore', '.ipynb_checkpoints', 'LICENSE', 'MNIST_EXPLORATION_WITH_XGBOOST.ipynb', 'README.md', 'sample_submission.csv', 'test.csv', 'train.csv']


In [37]:
df_train = pd.read_csv('../mnist_data_exploration/train.csv')
y = df_train.label.values
X = df_train.drop('label', axis=1).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
#df_train.head()

In [38]:
#print(X_train)
dtrain = xgboost.DMatrix(X_train, label=y_train)
dval = xgboost.DMatrix(X_val, label=y_val)

params = {
    'max_depth': 5,                 # the maximum depth of each tree
    'eta': 0.5,                     # the training step for each iteration
    'eta_decay': 0.9,
    'min_eta': 0.05,
    'silent': 0,                    # logging mode - quiet
    'objective': 'multi:softmax',   # multiclass classification using the softmax objective
    'num_class': 10,                 # the number of classes that exist in this datset
    'early_stopping_rounds': 10,
    'n_estimators': 1000
}  

def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', metrics.accuracy_score(labels, preds)

results = {}
model = xgboost.train(params, dtrain, num_boost_round=100, evals=[(dval, 'val')], evals_result=results, feval=eval_acc)

[0]	val-merror:0.180952	val-acc:0.819048
[1]	val-merror:0.129524	val-acc:0.870476
[2]	val-merror:0.110476	val-acc:0.889524
[3]	val-merror:0.097619	val-acc:0.902381
[4]	val-merror:0.087143	val-acc:0.912857
[5]	val-merror:0.080714	val-acc:0.919286
[6]	val-merror:0.072857	val-acc:0.927143
[7]	val-merror:0.069286	val-acc:0.930714
[8]	val-merror:0.065	val-acc:0.935
[9]	val-merror:0.06	val-acc:0.94
[10]	val-merror:0.057143	val-acc:0.942857
[11]	val-merror:0.055	val-acc:0.945
[12]	val-merror:0.051429	val-acc:0.948571
[13]	val-merror:0.050714	val-acc:0.949286
[14]	val-merror:0.050952	val-acc:0.949048
[15]	val-merror:0.048333	val-acc:0.951667
[16]	val-merror:0.047143	val-acc:0.952857
[17]	val-merror:0.044286	val-acc:0.955714
[18]	val-merror:0.04381	val-acc:0.95619
[19]	val-merror:0.040714	val-acc:0.959286
[20]	val-merror:0.041429	val-acc:0.958571
[21]	val-merror:0.040476	val-acc:0.959524
[22]	val-merror:0.039286	val-acc:0.960714
[23]	val-merror:0.039286	val-acc:0.960714
[24]	val-merror:0.036905

In [39]:
df_test = pd.read_csv('../mnist_data_exploration/test.csv')
#df_test.head()
x_test = df_test.values
dtest = xgboost.DMatrix(x_test)

In [40]:
pred_values = model.predict(dtest)
print(pred_values.shape)
pred_values[:2]

(28000,)


array([2., 0.], dtype=float32)

In [41]:
df_sub = pd.read_csv('../mnist_data_exploration/sample_submission.csv')
df_sub.head()
df_sub.Label = pred_values.astype(np.int32)
df_sub.head()
df_sub.to_csv('submission.csv', index=False)